In [1]:
import numpy as np
import matplotlib as pt
import sympy as smp

In [9]:
a, t = smp.symbols('a t', real=True)
the, phi = smp.symbols(r'\theta \phi', cls=smp.Function)
the = the(t)
phi = phi(t)
the_t = smp.diff(the,t)
the_tt = smp.diff(the_t,t)
phi_t = smp.diff(phi,t)
phi_tt = smp.diff(phi_t,t)

In [10]:
x = a * smp.sin(the) * smp.cos(phi)
y = a * smp.sin(the) * smp.sin(phi)
z = a * smp.cos(the)

In [13]:
m, g = smp.symbols('m g', real=True, positive=True)
T = smp.Rational(1,2) * m * (smp.diff(x,t)**2 + smp.diff(y,t)**2 + smp.diff(z,t)**2)
U = m * g * z
L = T - U

In [14]:
L.simplify()

a*m*(a*(sin(\theta(t))**2*Derivative(\phi(t), t)**2 + Derivative(\theta(t), t)**2) - 2*g*cos(\theta(t)))/2

In [19]:
ELthe = smp.diff(smp.diff(L,the_t),t) - smp.diff(L, the)
ELthe.simplify()

a*m*(-a*sin(2*\theta(t))*Derivative(\phi(t), t)**2/2 + a*Derivative(\theta(t), (t, 2)) - g*sin(\theta(t)))

In [20]:
ELphi = smp.diff(smp.diff(L,phi_t),t) - smp.diff(L, phi)
ELphi.simplify()

a**2*m*(sin(2*\theta(t))*Derivative(\phi(t), t)*Derivative(\theta(t), t) - cos(2*\theta(t))*Derivative(\phi(t), (t, 2))/2 + Derivative(\phi(t), (t, 2))/2)

In [21]:
sols = smp.solve([ELthe, ELphi], (the_tt, phi_tt))

In [40]:
sol_the_tt = sols[the_tt].expand()
sol_the_tt

sin(\theta(t))*cos(\theta(t))*Derivative(\phi(t), t)**2 + g*sin(\theta(t))/a

In [24]:
sols[phi_tt]

-2*Derivative(\phi(t), t)*Derivative(\theta(t), t)/tan(\theta(t))

Importante chequear que acá $\phi$ es cíclica, así que la segunda ecuación en realidad me complica de más. Me alcanza con mirar el momento conservado $p_{\phi}$

In [38]:
pphi = smp.diff(L,phi_t)

In [39]:
pphi.simplify()

a**2*m*sin(\theta(t))**2*Derivative(\phi(t), t)

Entonces lo que voy a hacer es sustituir esto en la ecuación de $\theta_{tt}$

In [62]:
p = smp.symbols(r'p_{\phi}')
phi_t2 = p/(a**2*m*smp.sin(the)**2)
sol_the_tt.subs(phi_t, phi_t2).simplify().expand()

g*sin(\theta(t))/a + p_{\phi}**2*cos(\theta(t))/(a**4*m**2*sin(\theta(t))**3)

Esto sin embargo no modifica mi expresión guardada en `sol_the_tt`, solo hace la sustitución necesaria para mostrarmelo por pantalla. Lo que tengo que hacer entonces es sobre escribir esa variable con la nueva expresión, tengo que hacer esa asignación.

In [63]:
sol_the_tt.simplify().expand()

sin(\theta(t))*cos(\theta(t))*Derivative(\phi(t), t)**2 + g*sin(\theta(t))/a

In [64]:
sol_the_tt = sol_the_tt.subs(phi_t, phi_t2).simplify().expand()

In [65]:
sol_the_tt

g*sin(\theta(t))/a + p_{\phi}**2*cos(\theta(t))/(a**4*m**2*sin(\theta(t))**3)

Ahora sí, tengo mi expresión deseada para $\ddot\theta$, solo queda resolver la ecuación diferencial numéricamente.

In [74]:
dwdt_f = smp.lambdify((t,g,m,a,p,the,the_t), sol_the_tt)
dthedt_f = smp.lambdify(the_t,the_t)

In [75]:
def dSdt(S,t,g,m,a,p):
    the, w = S
    return [
        dthedt_f(w),
        dwdt_f(t,g,m,a,p, the, w)
    ]

In [76]:
from scipy.integrate import odeint
import numpy as np

In [82]:
t_e = np.linspace(1,10,1001)
g_e = 9.81
m_e = 1
a_e = 2
p_e = 1
condinic = [0, np.pi/5]
ans = odeint(dSdt, y0= condinic, t=t_e, args=(g_e,m_e,a_e,p_e))

 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.0000000000000D+00   r2 =  0.0000000000000D+00
 intdy--  t (=r1) illegal        
      in above message,  r1 =  0.1000000000000D-01
      t not in interval tcur - hu (= r1) to tcur (=r2)       
      in above,  r1 =  0.0000000000000D+00   r2 =  0.0000000000000D+00
 intdy--  t (=r1) illegal        
      in above message,  r1 =  0.2000000000000D-01
      t not in interval tcur - hu (= r1) to tcur (=r2)       
      in above,  r1 =  0.0000000000000D+00   r2 =  0.0000000000000D+00
 lsoda--  trouble from intdy. itask = i1, tout = r1  
      in above message,  i1 =         1
      in above message,  r1 =  0.2000000000000D-01


In [81]:
ans[0].T

array([[0.        ,        nan,        nan, ..., 0.        , 0.        ,
        0.        ],
       [0.62831853,        nan,        nan, ..., 0.        , 0.        ,
        0.        ]])

Bueno ni idea porqué el `odeint` no me dejó resolver el sistema de ecuaciones. Me quedaré con las ganas de resolver el sistema pero por lo menos me quedo tranquilo que no es lo que me pedían tampoco.